In [1]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

## Helper Functions
Below are a few helper function to make converting between different image data types and formats.

In [2]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

## Haar Cascade Classifier
For this tutorial we will run a simple object detection algorithm called Haar Cascade on our images and video fetched from our webcam. OpenCV has a pre-trained Haar Cascade face detection model.

In [3]:
# initialize the Haar Cascade face detection model
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

## Webcam Videos
Running code on webcam video is a little more complex than images. We need to start a video stream using our webcam as input. Then we run each frame through our progam (Drowness detection).


In [5]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [6]:
import cv2
import PIL
from PIL import Image
import tensorflow as tf
import numpy as np


In [7]:
SIZE=224

In [100]:
from keras.models import load_model
VGG16_model=load_model('/content/drive/MyDrive/ai-iot-project/ai/weights/Vgg16_cropped_final.h5') ## Replace with the path of the weights
from PIL import Image

In [ ]:
!pip install mtcnn

In [102]:
import cv2
import PIL
from PIL import Image
import tensorflow as tf
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mtcnn import MTCNN

def crop_face_and_return(image):
    cropped_face = None
    detector = MTCNN()
    faces = detector.detect_faces(image)
    if faces:

      x, y, width, height = faces[0]['box']

      cropped_face = image[y:y+height, x:x+width]

    return cropped_face

In [ ]:
import time

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialze bounding box to empty
bbox = ''
count = 0
filenames=['/content/image_1','/content/image_2','/content/image_3','/content/image_4','/content/image_5']
predictions=[]
i=0
while True and i<5:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])
    cv2.imwrite(filenames[i]+'.jpg', img)


    image = cv2.imread(filenames[i]+'.jpg')
    cropped_face = crop_face_and_return(image)
    if cropped_face is not None:
    # Convert the NumPy array 'cropped_face' into a PIL Image
        pil_image = Image.fromarray(cropped_face,'RGB')

        pil_image = pil_image.resize((SIZE, SIZE))

        cropped_face = np.array(pil_image)

        image=tf.reshape(cropped_face , (1, SIZE,SIZE, 3))

        predictions.append(VGG16_model.predict(image))

    time.sleep(0.5)

    i+=1




In [119]:
predictions=[np.argmax(pr) for pr in predictions]

In [ ]:
predictions

### **VIEDO TEST**

In [ ]:
!pip install mtcnn

In [30]:
import cv2
import PIL
from PIL import Image
import tensorflow as tf
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from mtcnn import MTCNN

def crop_face_and_return(image):
    cropped_face = None
    detector = MTCNN()
    faces = detector.detect_faces(image)
    if faces:

      x, y, width, height = faces[0]['box']

      cropped_face = image[y:y+height, x:x+width]

    return cropped_face

In [ ]:
!pip install opendatasets

In [ ]:
!pip install pandas

In [ ]:
import opendatasets as od
import pandas as pd

od.download("https://www.kaggle.com/datasets/nikospetrellis/nitymed")

In [34]:
SIZE=224

In [112]:
from keras.models import load_model
VGG16_model=load_model('/content/drive/MyDrive/ai-iot-project/ai/weights/Vgg16_cropped_final.h5') ## Replace with the path of the weights
from PIL import Image

In [113]:
import matplotlib.image as mpimg

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Open the video file

video_path = '/content/nitymed/DSM_Dataset-HDTV720/Yawning/Male/HDTV720/P1043124_720.mp4'  # Replace with the path to your video file
video = cv2.VideoCapture(video_path)

# Set the frame rate and interval
frame_rate = video.get(cv2.CAP_PROP_FPS)
interval = int(frame_rate * 3)  # Capture frame every 3 seconds

# Initialize variables
frame_count = 0
image_count = 1

predictions=[]


while (True) and (image_count<6)  :
    # Read the next frame
    ret, frame = video.read()

    # Break the loop if no more frames are available
    if not ret:
        break

    frame_count += 1

    # Save image every 2 seconds
    if frame_count % interval == 0:
        # Save the image
        image_path = f'/content/frame_{image_count}.jpg'
        cv2.imwrite(image_path, frame)

        image = cv2.imread(image_path)
        cropped_face = crop_face_and_return(image)
        if cropped_face is not None:

          pil_image = Image.fromarray(cropped_face)

          pil_image = pil_image.resize((SIZE, SIZE))


          image=tf.reshape(pil_image, (1, SIZE,SIZE, 3))
          predictions.append(VGG16_model.predict(image))


        image_count += 1
# Release the video
video.release()

In [115]:
predictions =[np.argmax(pr) for pr in  predictions]

In [ ]:
predictions

## **Send the prediction**

In [ ]:
out="Safe."
for i in range (0,len(predictions)-1):
    if  (predictions[i]==1 and predictions[i+1] ==1) or ((predictions[i]==1 and predictions[i+1] ==2)or(predictions[i]==2 and predictions[i+1] ==1)):
      out="".join("DROWSY DRIVER, Wake up!!")
      break
    elif (predictions[i]==2 and predictions[i+1] ==2):
      out="".join("You are going to sleep, drink coffee please!!")
      break
    else :
      continue
print(out)

In [ ]:
!pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt #import the client1
broker_address="mqtt-dashboard.com" ### set your broker address here
#broker_address="iot.eclipse.org" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
client.publish("NTI/final",out)#publish ### set your topic name

In [ ]:
from google.colab import drive
drive.mount('/content/drive')